In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

### Reading the text data

In [10]:
text_data_sentence = pd.read_csv('textdatanew.csv', encoding='ISO-8859-1')

In [11]:
text_data_sentence.head(5)

,CollectionID,BiographyID,ParagraphNo,ParagraphText
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...
2,a001,bio04,3,Cleopatra might have responded with a brillian...
3,a001,bio04,4,Caesar was then above fifty years of age. His ...
4,a001,bio04,5,For three years Cleopatra reigned with little ...


### Reading the text features

In [44]:
text_features = pd.read_csv("text_features.csv", encoding='ISO-8859-1')

C:\Users\arvra\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12,15,16,22,23,24,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
text_features.head(2)

,CollectionID,BiographyID,ParagraphNo,sadness,joy,fear,disgust,anger,score,sentiment,...,Number,Organization,Person,PrintMedia,Quantity,Sport,SportingEvent,TelevisionShow,Time,Vehicle
0,a001,bio04,1,0.255896,0.558011,0.101166,0.111615,0.054668,0.290669,positive,...,NaN,NaN,Cleopatra,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001,bio04,2,0.171629,0.257088,0.173474,0.098726,0.267978,0.000000,neutral,...,NaN,Roman senate,"Cleopatra, Julius Caesar, Pompey, Ptolemy",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reading the Response file

In [12]:
bess_tags = pd.read_csv('CBW_Bess_tags_final2.csv')

In [13]:
bess_tags.head()

,Content,Event,Type,para no,URI,author,biographyID,collectionID,personaName,title
0,after,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
1,culmination,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
2,middle,name,stageOfLife,2.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
3,middle,name,stageOfLife,3.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
4,middle,name,stageOfLife,4.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...


In [14]:
bess_reponse = bess_tags.loc[:,['Content','Event','Type','para no','biographyID','collectionID']]

In [15]:
bess_reponse= bess_reponse.fillna(' ')

In [16]:
bess_reponse.loc[:,'Response'] = bess_reponse.loc[:,['Content','Event']].apply(lambda x: '_'.join(x),axis = 1)

In [17]:
top_10_events = bess_reponse[bess_reponse.Type.isin(['Event'])].Response.value_counts()[0:10]
top_10_events

city_locationSetting            1351
husband_agentType               1245
conversation_type               1118
nobleman, named_agentType        853
father_agentType                 818
lover, male, named_agentType     776
royalty, male_agentType          753
famous man_agentType             712
writing letter_type              667
sovereign, male_agentType        655
Name: Response, dtype: int64

In [18]:
bess_reponse.shape

(170663, 7)

In [19]:
bess_reponse.loc[:,['para no','biographyID','collectionID','Response']].head()

,para no,biographyID,collectionID,Response
0,1,bio04,a001,after_name
1,1,bio04,a001,culmination_name
2,2,bio04,a001,middle_name
3,3,bio04,a001,middle_name
4,4,bio04,a001,middle_name


In [20]:
bess_response_df = bess_reponse.loc[:,['para no','biographyID','collectionID','Response','Type']].drop_duplicates()

In [21]:
#bess_response_df.groupby(['para no','biographyID','collectionID','Type'],as_index = False)['Response'].apply(lambda x: ' '.join(x))

In [22]:
final_response_file = bess_response_df[bess_response_df.Response.isin(top_10_events.index)]

In [23]:
#final_response_file.melt()

In [24]:
final_response_file.head()

,para no,biographyID,collectionID,Response,Type
27,2,bio04,a001,"royalty, male_agentType",Event
29,2,bio04,a001,city_locationSetting,Event
35,3,bio04,a001,"royalty, male_agentType",Event
37,3,bio04,a001,city_locationSetting,Event
49,4,bio04,a001,"royalty, male_agentType",Event


In [25]:
final_response_file.loc[:,'values'] = 1

C:\Users\arvra\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\arvra\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
final_response_wide = (final_response_file.pivot_table(index=['para no','biographyID','collectionID','Type'],\
                                       columns='Response', values='values').reset_index())

In [27]:
final_response_wide = final_response_wide.fillna(0)

In [28]:
final_response_wide.head()

Response,para no,biographyID,collectionID,Type,city_locationSetting,conversation_type,famous man_agentType,father_agentType,husband_agentType,"lover, male, named_agentType","nobleman, named_agentType","royalty, male_agentType","sovereign, male_agentType",writing letter_type
0,1.0,bio01,a185,Event,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,bio01,a186,Event,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,bio01,a207,Event,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,bio01,a221,Event,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,bio01,a270,Event,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [29]:
text_data_sentence_final = pd.merge(text_data_sentence,final_response_wide, how = 'left', left_on = ['CollectionID','BiographyID','ParagraphNo'],\
                             right_on = ['collectionID','biographyID','para no'])

In [30]:
text_data_sentence_final = text_data_sentence_final[~text_data_sentence_final['para no'].isna()]

In [31]:
text_data_sentence_final_response = text_data_sentence_final[text_data_sentence_final.columns[-10:]]

In [32]:
final_response_file.Response.value_counts()

city_locationSetting            1180
conversation_type               1055
husband_agentType               1034
father_agentType                 672
writing letter_type              640
royalty, male_agentType          638
nobleman, named_agentType        630
famous man_agentType             618
lover, male, named_agentType     583
sovereign, male_agentType        526
Name: Response, dtype: int64

In [40]:
text_data_sentence_final.ParagraphText[2]

'Cleopatra might have responded with a brilliant retinue, or sent distinguished advocates to represent her. Instead, with a single companion, Apollodorus, she embarked in a fragile skiff and made a tempestuous voyage to Alexandria. Arrived there the question of how she should gain audience with Caesar became all important. That a queen should appear bedraggled from a sea voyage and accompanied by but one attendant was impossible. Cleopatra, whose dramatic instinct, no less than her personal charms, would be invaluable in the comic opera field to-day, solved the problem. Clad in raiment of which the least said the most descriptive, she lay upon a rich oriental rug and was wrapped up by the faithful Apollodorus into a neat bundle, which he thereupon shouldered and carried to the palace. Upon the plea of bringing to the Roman general a tribute from the absent queen of Egypt, he found prompt admission to the audience hall. There he laid his burden on the floor before Caesar, undid the fast

In [206]:
### Final data with the text features

text_features.drop(['ParagraphText'],axis = 1,inplace = True)
text_data_sentence_final_2 = pd.merge(text_data_sentence_final,text_features,how = 'left',
                                 on=['CollectionID', 'BiographyID', 'ParagraphNo'])

In [207]:
text_data_sentence_final_2.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,para no,biographyID,collectionID,Type,city_locationSetting,conversation_type,...,Number,Organization,Person,PrintMedia,Quantity,Sport,SportingEvent,TelevisionShow,Time,Vehicle
0,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,2.0,bio04,a001,Event,1.0,0.0,...,NaN,Roman senate,"Cleopatra, Julius Caesar, Pompey, Ptolemy",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001,bio04,3,Cleopatra might have responded with a brillian...,3.0,bio04,a001,Event,1.0,0.0,...,NaN,NaN,"Apollodorus, Caesar, Cleopatra",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a001,bio04,4,Caesar was then above fifty years of age. His ...,4.0,bio04,a001,Event,1.0,0.0,...,NaN,NaN,"Brutus, Caesar, Cleopatra, Julius, Ptolemy",NaN,fifty years,NaN,NaN,NaN,NaN,NaN
3,a001,bio04,5,For three years Cleopatra reigned with little ...,5.0,bio04,a001,Event,0.0,0.0,...,NaN,NaN,"Brutus, Caesar, Cassius, Cleopatra, Mark Anton...",NaN,three years,NaN,NaN,NaN,NaN,NaN
4,a001,bio04,6,For Antony now were all the amorous dalliances...,6.0,bio04,a001,Event,0.0,0.0,...,NaN,NaN,"Antony, Caesar, Cleopatra",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Text - Preprocessing

#### Getting stop words - High Frequency and Low Frequency word list

In [211]:
from collections import Counter
from nltk.tokenize import word_tokenize

In [215]:
tokenized_para = text_data_sentence_final_2.ParagraphText.apply(word_tokenize)

In [223]:
all_sent = [words for each_sent in tokenized_para for words in each_sent]

In [226]:
count_dict = Counter(all_sent)

In [237]:
high_freq_words = [word for (word,count) in count_dict.most_common(500)]

In [243]:
less_freq_words = []
threshold = 5

for k,v in count_dict.items():
    
    if v < threshold:
        less_freq_words.append(k)

#### Bag of Words

In [246]:
stop_words = stopwords.words('english')
stop_words.extend(high_freq_words)
stop_words.extend(less_freq_words)

In [250]:
bow_model = CountVectorizer(ngram_range= (1,2),stop_words=stop_words)

In [251]:
Para_text_bow = bow_model.fit_transform(text_data_sentence_final_2.ParagraphText)

#### Tf-IDF

In [252]:
tf_idf_model = TfidfVectorizer(ngram_range=(1,2),stop_words = stop_words)

In [253]:
Para_text_tfidf = tf_idf_model.fit_transform(text_data_sentence_final.ParagraphText)

### Combining the bag of words with other features

In [254]:
import scipy as sp

In [255]:
text_data_sentence_final_2.columns

Index(['CollectionID', 'BiographyID', 'ParagraphNo', 'ParagraphText',
       'para no', 'biographyID', 'collectionID', 'Type',
       'city_locationSetting', 'conversation_type', 'famous man_agentType',
       'father_agentType', 'husband_agentType', 'lover, male, named_agentType',
       'nobleman, named_agentType', 'royalty, male_agentType',
       'sovereign, male_agentType', 'writing letter_type', 'sadness', 'joy',
       'fear', 'disgust', 'anger', 'score', 'sentiment', 'Anatomy', 'Award',
       'Company', 'Crime', 'Date', 'Drug', 'Facility', 'GeographicFeature',
       'HealthCondition', 'JobTitle', 'Location', 'Measure', 'Movie', 'Number',
       'Organization', 'Person', 'PrintMedia', 'Quantity', 'Sport',
       'SportingEvent', 'TelevisionShow', 'Time', 'Vehicle'],
      dtype='object')

In [256]:
### Getting sentiment and emotional feautres

emotional_features = text_data_sentence_final_2.loc[:,'sadness':'score']
emotional_features = emotional_features.fillna(0)

## Converting Sentiment to codes
sentiment_cat_codes = pd.DataFrame(text_data_sentence_final_2['sentiment'].astype('category').cat.codes).values.shape

#### Getting the encoded version of spacy features
text_data_spacy_features = text_data_sentence_final_2.loc[:,'Anatomy':'Vehicle'].fillna(' ')
text_data_spacy_features_codes = text_data_spacy_features.apply(lambda x: x.astype('category').cat.codes)

In [257]:
X = sp.sparse.hstack((Para_text_bow, sentiment_cat_codes,
                      emotional_features,
                      text_data_spacy_features_codes.values) ,format='csr')

### Splitting data into train and test

In [258]:
from sklearn.model_selection import train_test_split

In [259]:
X_train, X_test, y_train, y_test = train_test_split(X ,text_data_sentence_final_response,
                                                    test_size = 0.3, random_state = 0)

In [260]:
features = bow_model.get_feature_names()
features.extend(['Sentiment'])
features.extend(emotional_features.columns.values)
features.extend(text_data_spacy_features_codes.columns.values)

### Machine Learning Models

In [271]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [309]:
#knn_model = KNeighborsClassifier(n_neighbors= 3, p = 1.5)
rf_model = RandomForestClassifier(n_estimators= 50)
lr_model = LogisticRegression(multi_class='multinomial')
#rf_model = SVC(C = 10, kernel = 'poly')

In [310]:
#y_train.melt()[y_train.melt().value.isin([1])].variable.value_counts()

In [311]:
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

#### Distribution of the response variables

In [312]:
pd.DataFrame([np.argmax(each) if each.sum() != 0 else 10 for each in rf_model.predict(X_test)])[0].value_counts()

10    1266
1      266
0       56
3        8
5        7
2        7
7        6
6        6
9        4
4        4
8        1
Name: 0, dtype: int64

In [283]:
from sklearn.metrics import confusion_matrix
preds = pd.DataFrame([np.argmax(each) if each.sum() != 0 else 10 for each in rf_model.predict(X_test)])[0]
#confusion_matrix(y_test,preds)

In [270]:
feature_importances = pd.DataFrame(rf_model.feature_importances_,
                                  index = features,
                                  columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(20)

,importance
Person,0.006898
sadness,0.006428
joy,0.006181
anger,0.004796
letter,0.004643
score,0.004511
Location,0.004486
fear,0.004065
disgust,0.003883
thou,0.003136


In [267]:
np.array(rf_model.predict_proba(X_test)).shape

(10, 1631, 2)

#### Getting the result by changing the threshold

In [268]:
result = [np.argmax([int(each[1]>0.4) for each in np.array(rf_model.predict_proba(X_test))[:,each_example,:]]) if \
     sum([int(each[1]>0.4) for each in np.array(rf_model.predict_proba(X_test))[:,each_example,:]]) > 0 else 10
            for each_example in range(1000)]

In [196]:
#result = [np.argmax(each) if each.sum() != 0 else 10 for each in rf_model.predict_proba(X_test)]

In [269]:
pd.DataFrame(result)[0].value_counts()

10    616
1     228
0      69
9      21
4      15
3      12
5      11
7       9
6       8
8       7
2       4
Name: 0, dtype: int64

In [205]:
y_test.melt()[y_test.melt().value.isin([1])].variable.value_counts()

city_locationSetting            328
conversation_type               302
husband_agentType               295
father_agentType                206
royalty, male_agentType         198
nobleman, named_agentType       196
writing letter_type             189
famous man_agentType            178
lover, male, named_agentType    168
sovereign, male_agentType       166
Name: variable, dtype: int64